In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_text as tf_text
import tensorflow_datasets as tfds

from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras import utils, layers, losses, Model, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import re 
import warnings
warnings.filterwarnings('ignore')

tf.__version__

2024-08-08 04:06:08.780205: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-08 04:06:08.999195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-08 04:06:09.180079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 04:06:09.348476: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 04:06:09.399050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 04:06:09.786197: I tensorflow/core/platform/cpu_feature_gu

'2.17.0'

In [3]:
from pathlib import Path
import pandas as pd
import random

df_path = Path("labelled_train_set.csv")
df = pd.read_csv(df_path)

labels = ["AI-generated", "Human-written"]

ai_text = []
human_text = []

train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

train_df.shape, test_df.shape

((450, 3), (50, 3))

In [4]:
data_dir = Path("data")
train_dir = data_dir / "train"
test_dir = data_dir / "test"

# data_dir.mkdir(parents=True, exist_ok=True)
# train_dir.mkdir(parents=True, exist_ok=True)
# test_dir.mkdir(parents=True, exist_ok=True)

# def create_data_dir(df, dir:Path):
#     for (id, article, Type) in df.values:
#         if Type == "AI-generated":
#             label_dir = dir / Type
#             label_dir.mkdir(parents=True, exist_ok=True)
            
#             with open(label_dir / f"{id}.txt", "w") as f:
#                 f.write(article)
#         elif Type == "Human-written":
#             label_dir = dir / Type
#             label_dir.mkdir(parents=True, exist_ok=True)
            
#             with open(label_dir / f"{id}.txt", "w") as f:
#                 f.write(article)

# create_data_dir(train_df, train_dir)
# create_data_dir(test_df, test_dir)

In [5]:
train_df['Type'].value_counts()

Type
Human-written    349
AI-generated     101
Name: count, dtype: int64

In [6]:
def clean(text):
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^a-z.?!\,\-\s]', '')
    text = tf.strings.regex_replace(text, '[.?!,¿]', ' ')
    text = tf.strings.strip(text)
    text = tf.strings.regex_replace(text, '\.\.\.', ' ')
    text = tf.strings.join(['', text, ''], separator=' ')
    return text

def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return tf_text.normalize_utf8(text, 'NFKD')

In [7]:
train_df['Article'][23]

'Compilers are traditionally thought of as having a â€œfront endâ€\x9d that analyzes the source code (high-level language statements) and a â€œback endâ€\x9d that generates the appropriate low-level code. The front end processing begins with lexical analysis. The compiler scans the source program looking for matches to valid tokens as defined by the language. A token is any word or symbol that has meaning in the language, such as a keyword (reserved word) such as if or while. Next, the tokens are parsed or grouped according to the rules of the language. The result of parsing is a â€œparse treeâ€\x9d that resolves statements into their component parts. For example, an assignment statement may be parsed into an identifier, an assignment operator (such as =), and a value to be assigned. The value in turn may be an arithmetic expression that consists of operators and operands.'

In [8]:
clean(train_df['Article'][23])

<tf.Tensor: shape=(), dtype=string, numpy=b' compilers are traditionally thought of as having a afront enda that analyzes the source code high-level language statements and a aback enda that generates the appropriate low-level code  the front end processing begins with lexical analysis  the compiler scans the source program looking for matches to valid tokens as defined by the language  a token is any word or symbol that has meaning in the language  such as a keyword reserved word such as if or while  next  the tokens are parsed or grouped according to the rules of the language  the result of parsing is a aparse treea that resolves statements into their component parts  for example  an assignment statement may be parsed into an identifier  an assignment operator such as   and a value to be assigned  the value in turn may be an arithmetic expression that consists of operators and operands '>

In [9]:
clean_text(train_df['Article'][23])

<tf.Tensor: shape=(), dtype=string, numpy=b'compilers are traditionally thought of as having a front end that analyzes the source code highlevel language statements and a back end that generates the appropriate lowlevel code the front end processing begins with lexical analysis the compiler scans the source program looking for matches to valid tokens as defined by the language a token is any word or symbol that has meaning in the language such as a keyword reserved word such as if or while next the tokens are parsed or grouped according to the rules of the language the result of parsing is a parse tree that resolves statements into their component parts for example an assignment statement may be parsed into an identifier an assignment operator such as and a value to be assigned the value in turn may be an arithmetic expression that consists of operators and operands'>

In [10]:
max_features = 75000 # 75000
embedding_dim = 64
sequence_length = 512*2

vectorizer_layer = TextVectorization(
    standardize=clean, # clean
    max_tokens=max_features,
    ngrams=(3, 5),
    output_mode='int',
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True
)

vectorizer_layer.adapt(train_df['Article'])
Text = vectorizer_layer(train_df['Article']).numpy()
print(Text.shape)
Text

(450, 1024)


array([[ 7771, 34054,     1, ...,     0,     0,     0],
       [    1,    75,   175, ...,     0,     0,     0],
       [    1,     1, 46834, ...,     0,     0,     0],
       ...,
       [73171,     1,    46, ...,     0,     0,     0],
       [26251,     1, 32532, ...,     0,     0,     0],
       [ 1280,     1, 62740, ...,     0,     0,     0]])

In [65]:
type_to_idx = {'Human-written': 0, 'AI-generated': 1}

train_df['target'] = train_df['Type'].map(type_to_idx)
test_df['target'] = test_df['Type'].map(type_to_idx)

In [66]:
sm = SMOTE(random_state=42)
Text, labels = sm.fit_resample(Text, train_df['target'])

In [67]:
X_train, X_test, y_train, y_test = train_test_split(Text, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    stratify=labels)
X_train.shape, X_test.shape

((558, 1024), (140, 1024))

In [75]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        
    def call(self, inputs, training=True):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layer_norm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layer_norm2(out1 + ffn_output)
    
inputs = Input(shape=(sequence_length, ), dtype=tf.int64)
x = Embedding(max_features, embedding_dim)(inputs)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
transformer_block = TransformerBlock(embedding_dim, 2, 32)
x = transformer_block(x)
x = Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_9 (Embedding)         │ (None, 1024, 64)       │     4,800,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 1024, 64)       │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_9             │ (None, 1024, 64)       │        37,664 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 340, 128)       │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_6          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,936,609 (18.83 MB)

 Trainable params: 4,936,609 (18.83 MB)

 Non-trainable params: 0 (0.00 B)

In [76]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True,show_dtype=True,dpi=120)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [77]:
checkpoint_cb = ModelCheckpoint('model_att.keras', save_best_only=True)
early_stopping_cb = EarlyStopping(patience=3, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5181 - loss: 0.9633 - val_accuracy: 0.9071 - val_loss: 0.4957
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8346 - loss: 0.4372 - val_accuracy: 0.9071 - val_loss: 0.2896
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9519 - loss: 0.1906 - val_accuracy: 0.8786 - val_loss: 0.3573
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9747 - loss: 0.1017 - val_accuracy: 0.8643 - val_loss: 0.3258
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9910 - loss: 0.0362 - val_accuracy: 0.8643 - val_loss: 0.4095


In [78]:
hist_ = pd.DataFrame(hist.history)
hist_

,accuracy,loss,val_accuracy,val_loss
0,0.568100,0.799055,0.907143,0.495681
1,0.851254,0.394908,0.907143,0.289626
2,0.956989,0.171428,0.878571,0.357310
3,0.985663,0.065578,0.864286,0.325841
4,0.994624,0.024514,0.864286,0.409546


In [79]:
predictions = model.predict(X_test)
y_pred = np.where(predictions > 0.5, 1, 0)

y_text_1d = np.ravel(y_test)
y_pred_1d = np.ravel(y_pred)
df_pred = pd.DataFrame({'Actual': y_text_1d, 'Prediction': y_pred_1d})
df_pred

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 306ms/step


,Actual,Prediction
0,0,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
135,1,0
136,1,0
137,0,0
138,1,1


In [80]:
cr = classification_report(y_text_1d, y_pred_1d)
print(cr)

              precision    recall  f1-score   support

           0       0.86      0.97      0.91        70
           1       0.97      0.84      0.90        70

    accuracy                           0.91       140
   macro avg       0.91      0.91      0.91       140
weighted avg       0.91      0.91      0.91       140



In [ ]:
# test_text = vectorize_layer(test['text']).numpy()
# predictions = model.predict(test_text)
# y_pred = np.where(predictions>=.5,1,0)
# sub=pd.DataFrame()
# sub['id'],sub['generated']=test['id'],predictions
# sub.to_csv('/kaggle/working/submission.csv', index=False)
# sub